In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import psycopg2
from llama_index.core import Settings
import re

/Users/jeongwon/Desktop/Chatbot/Law/lawchat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
Settings.embed_model = HuggingFaceEmbedding(
    model_name="jhgan/ko-sroberta-multitask"
)

In [ ]:
try:
    from_table_name = 'each_paragraph'
    to_table_name = 'data_each_paragraph'

    conn = psycopg2.connect(host='host', dbname='dbname', user='user', password='password', port=5432) #you need to add your information
    cursor = conn.cursor()
    query = "SELECT name, part_num, part_name, chap_num, char_name, sec_num, sec_name, para_num, para_name, art_num, art_name, content FROM {table} order by id".format(table=from_table_name)
    cursor.execute(query)
    rows = cursor.fetchall()
    i = 1

    for row in rows:
        filtered_tuple = tuple(element for element in row if element is not None) #None값을 필터링하는 코드
        filtered_text = ' '.join(filtered_tuple)
        filtered_text = re.sub('(name:|part_num:|part_name:|chap_num:|char_name:|sec_num:|sec_name:|para_num:|para_name:|art_num:|art_name:|content:|None,|\n)','',filtered_text)
        embedding = Settings.embed_model.get_text_embedding(filtered_text)
        try:
            sql = "INSERT INTO {to_table_name} (text, embedding)VALUES ('{text}', '{embedding}'::VECTOR(768))".format(to_table_name=to_table_name, text= filtered_text, embedding = embedding)
            cursor.execute(sql)
        except Exception as e:
            print("Update Error: ", e)
        i+=1    
        conn.commit()

except Exception as e:
    print(f"데이터베이스 연결에 실패했습니다: {e}")
finally:
    # 데이터베이스 연결 종료
    if conn:
        cursor.close()
        conn.close()
        print("PostgreSQL 연결이 종료되었습니다.")


PostgreSQL 연결이 종료되었습니다.
